In [15]:
from importlib import reload
import sys
sys.path.append('..')

import cartopy.crs as ccrs
import cartopy.feature as cfeature 
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import seaborn as sb
import xarray as xr
from datetime import datetime, timedelta
from itertools import chain

from src.gogoesgone import processing as pr
from src.gogoesgone import zarr_access as za
reload(pr)
reload(za)

<module 'src.gogoesgone.zarr_access' from '/home/m/m300931/gogoesgone/notebooks/../src/gogoesgone/zarr_access.py'>

# Save GOES-16 images for cold pool labelling in Zooniverse

This notebook accesses GOES-16 data via zarr, maps them onto a lon/lat grid, and saves them as .png images. This is done with the goal of uploading these images to Zooniverse, and labelling cold pool cloud holes from them. Periods are selected based on availability of in-situ observations, and the domain corresponds to a small region around Barbados which includes the HALO circles.

## Define periods for which we want to download data

The periods in this example correspond roughly to the times when the Raman lidars, cloud radars, and wind lidars were running at the Barbados Cloud Observatory. Choose only dayl

In [2]:
period_1 = ["20150915 00:00:00", "20160915 00:00:00"]
period_2 = ["20161004 00:00:00", "20161022 00:00:00"]
period_3 = ["20161101 00:00:00", "20161130 00:00:00"]
period_4 = ["20161208 00:00:00", "20170120 00:00:00"]
period_5 = ["20170125 00:00:00", "20170216 00:00:00"]
period_6 = ["20170315 00:00:00", "20171022 00:00:00"]
period_7 = ["20171101 00:00:00", "20171210 00:00:00"]
period_8 = ["20180103 00:00:00", "20180722 00:00:00"]
period_9 = ["20180103 00:00:00", "20180722 00:00:00"]
period_10 = ["20181219 00:00:00", "20210125 00:00:00"]
period_11 = ["20220925 00:00:00", "20230315 00:00:00"]
period_12 = ["20230427 00:00:00", "20230615 00:00:00"]

periods = [period_1, period_2, period_3, period_4, period_5, period_6, period_7, period_8, period_9, period_10, period_11]

In [17]:
test_period = ["20200124 00:00:00", "20200125 00:00:00"]
test_flist = za.periods_url(test_period, hour_range=[10,21])

## Access data for all periods

In [84]:
flist = []
channel=2

for period in periods:
    flist.append(za.periods_url(period))
    
flist = sorted(list(chain(*flist)))
m = za.get_mapper_from_mzz(flist)

KeyboardInterrupt: 

In [4]:
time = "20200124 00:00:00"
format = "%Y%m%d %H:%M:%S"
ch = 2 

flist= za.nearest_time_url(time,format, channel=ch)
print('got list of url')
m = za.get_mapper_from_mzz(flist)
print('mapper done')

got list of url


/home/m/m300931/.conda/envs/gogoesgone_env/lib/python3.10/site-packages/kerchunk/combine.py:269: UserWarning: Concatenated coordinate 't' contains less than expectednumber of values across the datasets: [6.33096302e+08]
  warnings.warn(


mapper done


In [43]:
time = "20200124"# 16:00:00"
format = "%Y%m%d"# %H:%M:%S"
ch = 13 

flist= za.nearest_time_url(time,format, channel=ch)
print('got list of url')
m = za.get_mapper_from_mzz(flist)
print('mapper done')

got list of url
mapper done


/home/m/m300931/.conda/envs/gogoesgone_env/lib/python3.10/site-packages/kerchunk/combine.py:269: UserWarning: Concatenated coordinate 't' contains less than expectednumber of values across the datasets: [6.33096302e+08]
  warnings.warn(


In [5]:
img = pr.Image(m)
extent = (-61,-53,11,16)

subset = img.subset_region_from_latlon_extents(extent, unit="degree")

/home/m/m300931/gogoesgone/notebooks/../src/gogoesgone/processing.py:28: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  self.dataset = xr.open_dataset(filepath, engine="zarr")
/home/m/m300931/gogoesgone/notebooks/../src/gogoesgone/processing.py:178: RuntimeWarning: invalid value encountered in sqrt
  r_s = (-b - np.sqrt(b**2 - (4 * a * c))) / (2 * a)


In [6]:
subset

<xarray.Dataset>
Dimensions:                                           (t: 1, y: 1069, x: 1678,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       band: 1)
Coordinates:
    band_id                                           (t, band) int8 ...
    band_wavelength                                   (t, band) float32 ...
  * t                                                 (t) datetime64[ns] 2020...
  * x                                                 (x) float64 0.0409 ... ...
    x_image                                           (t) float32 ...
  * y                                                 (y) float64 0.0482 ... ...
    y_image                                           (t) float32 ...
    lat                                               (y, x) float64 16.0 ......
    lon                                               (y, x) float64 -61.0 .....
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/33)
    CMI                                               (t, y, x) float32 0.0 ....
    DQF                                               (t, y, x) float32 0.0 ....
    algorithm_dynamic_input_data_container            (t, y, x) float64 nan ....
    algorithm_product_version_container               (t, y, x) float64 nan ....
    earth_sun_distance_anomaly_in_AU                  (t, y, x) float32 0.984...
    esun                                              (t, y, x) float32 1.631...
    ...                                                ...
    std_dev_reflectance_factor                        (t, y, x) float32 0.095...
    time_bounds                                       (t, number_of_time_bounds, y, x) datetime64[ns] ...
    total_number_of_points                            (t, y, x) float64 3.687...
    valid_pixel_count                                 (t, y, x) float64 3.687...
    x_image_bounds                                    (t, number_of_image_bounds, y, x) float32 ...
    y_image_bounds                                    (t, number_of_image_bounds, y, x) float32 ...
Attributes: (12/29)
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Image
    dataset_name:              OR_ABI-L2-CMIPF-M6C02_G16_s20200240000163_e202...
    date_created:              2020-01-24T00:09:55.4Z
    id:                        0a1c7056-76cc-427b-9094-93dce36ddeb4
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    summary:                   Single reflective band Cloud and Moisture Imag...
    time_coverage_end:         2020-01-24T00:09:47.1Z
    time_coverage_start:       2020-01-24T00:00:16.3Z
    timeline_id:               ABI Mode 6
    title:                     ABI L2 Cloud and Moisture Imagery

In [1]:
sb.set_context('paper')
fig = plt.figure(figsize=(8,5))

ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im = subset.CMI.isel(t=3).plot(ax=ax,x='lon',y='lat',cmap='Greys_r',add_colorbar=False,vmin=0,vmax=0.17)
#subset.CMI.isel(t=0).plot.contour(ax=ax,x='lon',y='lat',cmap='Reds',add_colorbar=False,levels=[290,],linewidths=.5)          
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='red', linewidth=1)

#Add title
plt.title(f"{subset.attrs['time_coverage_start']} to {subset.attrs['time_coverage_end']}")
#plt.colorbar(im,orientation='horizontal',label=f'Channel-{ch} Brightness Temperature / K')

'''
# # # Draw grid.
gl = ax.gridlines(ccrs.PlateCarree(), linewidth = 1, color = 'w', alpha = 0.5, linestyle=':', draw_labels = True)
gl.top_labels = False
gl.right_labels = False
gl.xlines = True
gl.ylines = True
gl.xlocator = mticker.FixedLocator(np.arange(-180, 180, 2.5))
gl.ylocator = mticker.FixedLocator(np.arange(-90, 90, 2.5))
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'color': 'grey'}
gl.ylabel_style = {'color': 'grey'}
'''

NameError: name 'sb' is not defined